In [ ]:
import pandas as pd
import numpy as np
from matplotlib_venn import venn3
from matplotlib_venn import venn2
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from cycler import cycler
%matplotlib inline

## First load the HES datasets

In [ ]:
location = 'C:/Users/Andrew Devereau/Documents/GeL/secondary data/Data applications/HES return October 2016/'
filename = 'NIC12784_AE.txt'

In [ ]:
dataAE = pd.read_csv(location+filename, sep='|')  #this is the A&E data set

In [ ]:
filename = 'NIC12784_CC.txt'  #Critical care dataset
dataCC = pd.read_csv(location+filename, sep='|')

In [ ]:
filename = 'NIC12784_APC.txt'   #this is the admitted patient care dataset
dataAPC = pd.read_csv(location+filename, sep='|')

In [ ]:
filename = 'NIC12784_OP.txt'   #this is the outpatient dataset
dataOP = pd.read_csv(location+filename, sep='|')

### Get the cancer participants

In [ ]:
filename = 'Cancer.csv'
cancerIDs = pd.read_csv(location+filename, sep=',')  #get the set of cancer patients sent to HES

In [ ]:
cancerIDs = cancerIDs.rename(columns = {'Participant Id':'STUDY_ID'})  #rename the participant Id to STUDY_ID to match results

### Load the ICD, AE and specialty code lookup files and define lookup functions

In [ ]:
filename = 'Tabular.xml'       #get ICD10 code list to convert codes to names
ICDtree = ET.parse(location+filename)
ICDroot = ICDtree.getroot()

In [ ]:
def getICD(search):  #look up ICD disease name from a search code
    for code in ICDroot.iter('diag'):
        name = code.find('name').text
        if name == search:
            desc = code.find('desc').text
            return(search + ' ' + desc)

In [ ]:
filename = 'ae.txt'
aecode = {}
with open(location+filename) as f:
    for line in f:
       (key, val) = line.split('\t')
       aecode[int(key)] = val.strip()

In [ ]:
def getAE(search): #look up diag code in AE data 
    try:
        return (aecode[int(search)])
    except:
        return('None')

In [ ]:
filename = 'spefCode.txt'
spefCode = {}
with open(location+filename) as f:
    for line in f:
       (key, val) = line.split('\t')
       spefCode[key] = val.strip()

In [ ]:
def getSpef(search): #look up speciality codes
    try:
        return (spefCode[search])
    except:
        return('None')

In [ ]:
def getNo (prefix, max): #generate a serial number string up to max with prefix, e.g. 'OPERTN_01..24'
    for no in range (1,max+1):
        if no < 10:
            yield (prefix + '0' + str(no))
        else:
            yield (prefix + str(no))

### merge CC with APC to match study id with susrecid

In [ ]:
CCmerge = pd.merge(dataCC, dataAPC, on='SUSRECID', how = 'left')   

### Make cancer subsets of the HES data sets

In [ ]:
cancerCC = CCmerge[(CCmerge['STUDY_ID'].isin(cancerIDs['STUDY_ID']))] 

In [ ]:
cancerCC.info(verbose=True, null_counts=True)

In [ ]:
cancerAE = dataAE[(dataAE['STUDY_ID'].isin(cancerIDs['STUDY_ID']))] 

In [ ]:
cancerAE.info()

In [ ]:
len(cancerAE['STUDY_ID'].value_counts())

In [ ]:
cancerAE.info(verbose=True, null_counts=True)

In [ ]:
cancerAPC = dataAPC[(dataAPC['STUDY_ID'].isin(cancerIDs['STUDY_ID']))]   #get cancer patients from APC results

In [ ]:
cancerAPC.info()

In [ ]:
len(cancerAPC['STUDY_ID'].value_counts())

In [ ]:
cancerAPC.info(verbose=True, null_counts=True)

In [ ]:
cancerOP = dataOP[(dataOP['STUDY_ID'].isin(cancerIDs['STUDY_ID']))]   #get cancer patients from OP results

In [ ]:
cancerOP.info(verbose=True, null_counts=True)

### Add consent dates to the cancer data sets

In [ ]:
filename = 'cancer_consent_2016-11-28_16-34-24.xlsx'
consentDates = pd.read_excel(location+filename)  #get consent dates

In [ ]:
consentDates = consentDates.drop_duplicates('Participant Identifiers Id', keep='first')  #remove duplicate records

In [ ]:
consentDates.drop(['Metadata Date', 'Event Date', 'Consent Given Id'], axis=1, inplace=True) #remove all columns except participant ID and date

In [ ]:
consentDates.rename(columns={'Participant Identifiers Id': 'STUDY_ID'}, inplace=True) #rename participant to STUDY_ID

In [ ]:
consentDates.head()

In [ ]:
cancerIDs[~(cancerIDs['STUDY_ID'].isin(consentDates['STUDY_ID']))]  #check for any cancerIDs not in the consentDates set

In [ ]:
cancerCC = pd.merge(cancerCC, consentDates, on='STUDY_ID', how = 'left')

In [ ]:
cancerAE = pd.merge(cancerAE, consentDates, on='STUDY_ID', how = 'left')
cancerAPC = pd.merge(cancerAPC, consentDates, on='STUDY_ID', how = 'left')
cancerOP = pd.merge(cancerOP, consentDates, on='STUDY_ID', how = 'left')

### Look for radiotherapy using Specialty codes 370 and 800 in APC and OP

In [ ]:
def convertSpef (spef):  #some spef codes are integers or floats- convert to strings
    if type(spef) == int:
        return str(spef)
    elif type(spef) == float:
        return str(int(spef))
    else:
        return spef

In [ ]:
spefList = ['370', '800', 370, 800]  #list of speciality codes to indicate radiotherapy. Both int and string values are present

In [ ]:
oncologyAPCPatients = cancerAPC[((cancerAPC.MAINSPEF.isin(spefList)) | (cancerAPC.TRETSPEF.isin(spefList))) & (pd.to_datetime(cancerAPC.ADMIDATE) < cancerAPC['Date Of Consent'])]

In [ ]:
len(oncologyAPCPatients['STUDY_ID'].unique())

In [ ]:
oncologyOPPatients = cancerOP[((cancerOP.MAINSPEF.isin(spefList))|(cancerOP.TRETSPEF.isin(spefList))) & (pd.to_datetime(cancerOP.APPTDATE) < cancerOP['Date Of Consent'])]

In [ ]:
len(oncologyOPPatients['STUDY_ID'].unique())

In [ ]:
columnVals = ['STUDY_ID', 'ADMIDATE', 'MAINSPEF', 'TRETSPEF', 'APPTDATE', 'Date Of Consent']  #create subsets from APC and OP for just these columns
APCsubset = oncologyAPCPatients.loc[:, oncologyAPCPatients.columns.isin(columnVals)]
OPsubset = oncologyOPPatients.loc[:, oncologyOPPatients.columns.isin(columnVals)]
APCsubset = APCsubset.rename(columns = {'ADMIDATE':'DATE'}) #rename the different date fields to 'DATE'
OPsubset = OPsubset.rename(columns = {'APPTDATE':'DATE'}) 

In [ ]:
oncologyHES = pd.concat([APCsubset, OPsubset])  #make a single subset of participants

In [ ]:
oncologyHES['TRETSPEF'].value_counts().index

In [ ]:
oncologyHES['MAINSPEF'] = oncologyHES['MAINSPEF'].apply(convertSpef)
oncologyHES['TRETSPEF'] = oncologyHES['TRETSPEF'].apply(convertSpef)

In [ ]:
len(oncologyHES['STUDY_ID'].unique()) #189 pariticpants have an oncology specialty history

In [ ]:
def getNewID (ID, IDDict):   #function to create new participant ID for plotting purposes
    return IDDict.get(ID)

In [ ]:
IDs = list(np.sort(oncologyHES['STUDY_ID'].unique()))
newIDs = list(int(x) for x in range(1,len(IDs)))
zipped = zip(IDs, newIDs)
IDDict = dict(zip(IDs, newIDs))   #create a dictionary mapping STUDY_ID to a new smaller ID

In [ ]:
oncologyHES['ID'] = oncologyHES['STUDY_ID'].apply(getNewID, args =(IDDict,))  #add new column with new ID

In [ ]:
groups1 = oncologyHES.groupby('MAINSPEF')   #create a timeseries plot grouped by specialty code
groups2 = oncologyHES.groupby('TRETSPEF')
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
ax.set_ylabel('Patient number')
for name, group in groups2:
    if name in ['370', '800']:
        ax.plot(pd.to_datetime(group['DATE']), group['ID'], label = name, linestyle='none', marker='o', color = 'r', alpha=1)
    else:
        ax.plot(pd.to_datetime(group['DATE']), group['ID'], label = name, linestyle='none', marker='.', color = 'b', alpha=1)
for name, group in groups1:
    if name in ['370', '800']:
        ax.plot(pd.to_datetime(group['DATE']), group['ID'], label = name, linestyle='none', marker='o', color = 'r', alpha=1)
    else:
        ax.plot(pd.to_datetime(group['DATE']), group['ID'], label = name, linestyle='none', marker='.', color = 'b', alpha=1)
#ax.legend(loc='best')

plt.show()

In [ ]:
mainSpef = oncologyHES['MAINSPEF'].value_counts().sort_values(ascending=False).to_frame()

In [ ]:
mainSpef['Specialty'] = mainSpef.index.map(getSpef)

In [ ]:
mainSpef  #these are the specialties with frequencies

In [ ]:
tretSpef = oncologyHES['TRETSPEF'].value_counts().sort_values(ascending=False).to_frame()
tretSpef['Specialty'] = tretSpef.index.map(getSpef)

In [ ]:
tretSpef

In [ ]:
oncPatients = pd.DataFrame(oncologyHES.STUDY_ID.value_counts().index)

In [ ]:
oncPatients.to_excel('oncologyHES.xlsx', index=False)  #save a list of the oncology patients